In [ ]:
# mike babb
# 2025 02 26
# simulate bingo

In [ ]:
# standard
import os

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

sns.set_theme(style="ticks")


In [ ]:
# the typical bingo grid is 25 squares (24 playable, 1 free play)

In [ ]:
def get_array_length(arr1:np.array, arr2:np.array):
    return len(set(arr1).intersection(arr2))

In [ ]:
def do_a_bingo_run(n_samples:int, n_trials:str, output_file_path:str):
    trial_list = []
    for nt in range(0, n_trials + 1):        

        # build the board
        numbers = list(range(1, n_samples + 1))
        # shuffle the numbers
        np.random.shuffle(numbers)
        # draw 24 numbers, insert 0 into the center, reshape the board to be a 5,5 matrix
        # this is the bingo board
        bb = np.insert(arr = np.array(numbers[:24]), obj = 12, values = 0).reshape(5,5)
        
        # shuffle the full list of numbers again to simulate drawing
        np.random.shuffle(numbers)
        numbers = np.insert(arr = numbers, obj = 0, values = 0)

        # free-play / 0 square is always already drawn        
        keep_drawing = True 
        # start counting
        i_nn = 1
        while keep_drawing:
            curr_list = numbers[:i_nn]
            # create a list of numbers drawn
                  
            # check horizontal and vertical slices
            for my_index in range(0, 5):        
                # rows        
                if get_array_length(arr1=bb[my_index, :], arr2=curr_list) >= 5:
                    trial_list.append([nt, i_nn, 0, my_index])
                    keep_drawing = False                
                # columns
                if get_array_length(arr1=bb[:, my_index], arr2=curr_list) >= 5:
                    trial_list.append([nt, i_nn, 1, my_index])
                    keep_drawing = False                
            # diagonal
            if get_array_length(arr1=bb.diagonal(), arr2=curr_list) >= 5:
                trial_list.append([nt, i_nn, 2, 1])
                keep_drawing = False
                
            # off diagonal
            if get_array_length(arr1=np.fliplr(m = bb).diagonal(), arr2=curr_list) >= 5:
                trial_list.append([nt, i_nn, 2, 2])
                keep_drawing = False
            i_nn += 1

    columns = ['trial_number', 'n_draws', 'card_part', 'idx']
    outcome = pd.DataFrame(data = trial_list, columns = columns)
    # subtract one to account for the first few draws where it's impossible
    # to get a bingo
    outcome['n_draws'] = outcome['n_draws'] - 1
    
    print(outcome['n_draws'].min())

    output_file_name = f'bingo_sim_{str(n_trials).zfill(6)}_{str(n_samples).zfill(3)}_.csv'
    ofpn = os.path.join(output_file_path, output_file_name)
    outcome.to_csv(path_or_buf= ofpn, sep = '\t', index = False)

    return outcome

In [ ]:
output_file_path = 'H:/project/bingo_simulator/data/'
n_trials = 10000
for ns in range(25, 51):
    print(ns)
    df = do_a_bingo_run(n_samples = ns, n_trials = n_trials, output_file_path = output_file_path)

In [ ]:
df['card_part'].value_counts()